In [10]:
import numpy as np
import scipy.linalg as la
import mne.filter as bandpass_filter
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import geomstats as gs
import geomstats.geometry.spd_matrices as spd
from geomstats.learning.frechet_mean import FrechetMean
from scipy.io import loadmat  

In [11]:
class RCSP:
    def __init__(self, channels):
        self.channels = channels
        self.V=None
      
    def bandpass(self, X, fs, fl, fh):
        X = np.swapaxes(X, 1, 2)
        X = bandpass_filter.filter_data(X, fs, fl, fh, verbose=False)
        X = np.swapaxes(X, 1, 2)
        return X
        
    def calculateCovariances(self, X):
        covlist = np.empty((X.shape[0], X.shape[2], X.shape[2]))
        for i in range(len(X)):
            epoch = X[i]
            covlist[i] = np.dot(epoch.T, epoch)
        return covlist
    
    def estimateMeans(self, classSpecificCOV, metric):
        if metric=="AIRM":
            estimator = FrechetMean(spd.SPDMetricAffine(n=self.channels), max_iter=64)
        elif metric=="LEM":
            estimator = FrechetMean(spd.SPDMetricLogEuclidean(n=self.channels), max_iter=64)
        elif metric=="BW":
            estimator = FrechetMean(spd.SPDMetricBuresWasserstein(n=self.channels), max_iter=64) #doesn't work yet
        else:
            raise Exception("Not implemented metric")
            
        means = []
        
        for COV in classSpecificCOV:
            estimator.fit(COV)
            mean = estimator.estimate_
            means.append(mean)
        return means
    
    
    def separate_classes(self, X, Y):
        classSpecificCOV = []
        for i in range(2): 
            ind = np.where(Y==i)[0]
            classCOV = X[ind]
            classSpecificCOV.append(classCOV)
        return classSpecificCOV
    
    def CSP(self, X, Y, metric, n):
        classSpecificCOV = self.separate_classes(X, Y)
        if metric == "classic":
            class0_avg = sum(classSpecificCOV[0])/len(classSpecificCOV[0])
            class1_avg = sum(classSpecificCOV[1])/len(classSpecificCOV[1])
        else:
            class0_avg, class1_avg = self.estimateMeans(classSpecificCOV, metric)

        _,V = la.eigh(class0_avg, class0_avg+class1_avg)
        V = np.concatenate((V[:, :n], V[:, -n:]), axis=1)
        
        return V
    
    def applyCSP(self, dk, V):
        a = np.dot(np.dot(V.T, dk), V) 
        f = np.log(np.diagonal(a)/np.trace(a)) 
        return f
    
    def get_mi_features(self, data, metric="classic", mode="train", fs=1000, fl=8, fh=30, n=3):

        channels = [7, 8, 9, 10, 12, 13, 14, 17, 18, 19, 20, 32, 33, 34, 35, 36, 37, 38, 39, 40] #standard 10/20 cap mapping, electrodes responsible for MI

        X = data['smt'][0][0][1000:3500, :, channels]
        Y = data['y_dec'][0][0][0]

        X = np.swapaxes(X, 0, 1)
        Y = Y-1

        filteredX = self.bandpass(X, fs, fl, fh)
        print(filteredX.shape)
        covX = self.calculateCovariances(filteredX)
        
        if mode=="train":
            V = self.CSP(covX, Y, metric, n)
            self.V = V
        else:
            if self.V is None:
                raise Exception("train CSP first please")
            V=self.V
        
        features = np.empty((len(X), 2*n))
  
        for i in range(len(X)):
            epoch = covX[i]
            features[i] = self.applyCSP(epoch, V) 
            
        return features, Y

In [12]:
csp = RCSP(20)

In [13]:
metrics = ["classic", "AIRM", "LEM"]
ns = [2,3,4,5]

for n in ns:
    print("N is " + str(n) + "\n")
    for metric in metrics:
        
        print("Metric is " + metric)
        av = []
        for i in range(1,21):        
            filename = 'datasets/54subjects/sess01/subj{}_EEG_MI.mat'.format(i)
            data = loadmat(filename)
            train = data['EEG_MI_train']
            test = data['EEG_MI_test']
    
            trainX, trainY = csp.get_mi_features(train, metric=metric, mode="train", n=n)
            testX, testY = csp.get_mi_features(test, metric=metric, mode="test", n=n)
            clf = LDA()
            clf.fit(trainX, trainY)
            res = clf.score(testX, testY)
            av.append(res)
            print("Subject " + str(i) + ", accuracy = " + str(res))
        print("Average is " + str(sum(av)/len(av)) + "\n")

N is 2

Metric is classic
(100, 2500, 20)
(100, 2500, 20)
Subject 1, accuracy = 0.6
(100, 2500, 20)
(100, 2500, 20)
Subject 2, accuracy = 0.95
(100, 2500, 20)
(100, 2500, 20)
Subject 3, accuracy = 0.95
(100, 2500, 20)
(100, 2500, 20)
Subject 4, accuracy = 0.55
(100, 2500, 20)
(100, 2500, 20)
Subject 5, accuracy = 0.92
(100, 2500, 20)
(100, 2500, 20)
Subject 6, accuracy = 0.78
(100, 2500, 20)
(100, 2500, 20)
Subject 7, accuracy = 0.5
(100, 2500, 20)
(100, 2500, 20)
Subject 8, accuracy = 0.55
(100, 2500, 20)
(100, 2500, 20)
Subject 9, accuracy = 0.83
(100, 2500, 20)
(100, 2500, 20)
Subject 10, accuracy = 0.56
(100, 2500, 20)
(100, 2500, 20)
Subject 11, accuracy = 0.45
(100, 2500, 20)
(100, 2500, 20)
Subject 12, accuracy = 0.49
(100, 2500, 20)
(100, 2500, 20)
Subject 13, accuracy = 0.56
(100, 2500, 20)
(100, 2500, 20)
Subject 14, accuracy = 0.57
(100, 2500, 20)
(100, 2500, 20)
Subject 15, accuracy = 0.51
(100, 2500, 20)
(100, 2500, 20)
Subject 16, accuracy = 0.5
(100, 2500, 20)
(100, 2500

(100, 2500, 20)
Subject 14, accuracy = 0.52
(100, 2500, 20)
(100, 2500, 20)
Subject 15, accuracy = 0.52
(100, 2500, 20)
(100, 2500, 20)
Subject 16, accuracy = 0.54
(100, 2500, 20)
(100, 2500, 20)
Subject 17, accuracy = 0.71
(100, 2500, 20)
(100, 2500, 20)
Subject 18, accuracy = 0.9
(100, 2500, 20)
(100, 2500, 20)
Subject 19, accuracy = 0.83
(100, 2500, 20)
(100, 2500, 20)
Subject 20, accuracy = 0.56
Average is 0.6639999999999999

Metric is AIRM
(100, 2500, 20)
(100, 2500, 20)
Subject 1, accuracy = 0.71
(100, 2500, 20)
(100, 2500, 20)
Subject 2, accuracy = 0.93
(100, 2500, 20)
(100, 2500, 20)
Subject 3, accuracy = 0.95
(100, 2500, 20)
(100, 2500, 20)
Subject 4, accuracy = 0.46
(100, 2500, 20)
(100, 2500, 20)
Subject 5, accuracy = 0.86
(100, 2500, 20)
(100, 2500, 20)
Subject 6, accuracy = 0.67
(100, 2500, 20)
(100, 2500, 20)
Subject 7, accuracy = 0.45
(100, 2500, 20)
(100, 2500, 20)
Subject 8, accuracy = 0.6
(100, 2500, 20)
(100, 2500, 20)
Subject 9, accuracy = 0.8
(100, 2500, 20)
(100, 

In [4]:
csp = RCSP(20)

In [4]:
metrics = ["classic", "AIRM", "LEM"]
ns = [2,3,4,5]

for n in ns:
    print("N is " + str(n) + "\n")
    for metric in metrics:
        
        print("Metric is " + metric)
        av = []
        for i in range(1,21):        
            filename = 'datasets/54subjects/sess01/subj{}_EEG_MI.mat'.format(i)
            data = loadmat(filename)
            train = data['EEG_MI_train']
            test = data['EEG_MI_test']
    
            trainX, trainY = csp.get_mi_features(train, metric=metric, mode="train", n=n)
            testX, testY = csp.get_mi_features(test, metric=metric, mode="test", n=n)
            clf = LDA()
            clf.fit(trainX, trainY)
            res = clf.score(testX, testY)
            av.append(res)
            print("Subject " + str(i) + ", accuracy = " + str(res))
        print("Average is " + str(sum(av)/len(av)) + "\n")

N is 2

Metric is classic
Subject 1, accuracy = 0.6
Subject 2, accuracy = 0.95
Subject 3, accuracy = 0.95
Subject 4, accuracy = 0.55
Subject 5, accuracy = 0.92
Subject 6, accuracy = 0.78
Subject 7, accuracy = 0.5
Subject 8, accuracy = 0.55
Subject 9, accuracy = 0.83
Subject 10, accuracy = 0.56
Subject 11, accuracy = 0.45
Subject 12, accuracy = 0.49
Subject 13, accuracy = 0.56
Subject 14, accuracy = 0.57
Subject 15, accuracy = 0.51
Subject 16, accuracy = 0.5
Subject 17, accuracy = 0.72
Subject 18, accuracy = 0.92
Subject 19, accuracy = 0.81
Subject 20, accuracy = 0.56
Average is 0.664

Metric is AIRM
Subject 1, accuracy = 0.71
Subject 2, accuracy = 0.95
Subject 3, accuracy = 0.95
Subject 4, accuracy = 0.42
Subject 5, accuracy = 0.92
Subject 6, accuracy = 0.69
Subject 7, accuracy = 0.56
Subject 8, accuracy = 0.56
Subject 9, accuracy = 0.85
Subject 10, accuracy = 0.65
Subject 11, accuracy = 0.52
Subject 12, accuracy = 0.45
Subject 13, accuracy = 0.54
Subject 14, accuracy = 0.51
Subject 15

In [6]:
metrics = ["classic", "AIRM", "LEM"]
ns = [2,3,4,5]

for n in ns:
    print("N is " + str(n) + "\n")
    for metric in metrics:
        
        print("Metric is " + metric)
        av = []
        for i in range(1,21):        
            filename = 'datasets/54subjects/sess01/subj{}_EEG_MI.mat'.format(i)
            data = loadmat(filename)
            train = data['EEG_MI_train']
            test = data['EEG_MI_test']
    
            trainX, trainY = csp.get_mi_features(train, metric=metric, mode="train", n=n)
            testX, testY = csp.get_mi_features(test, metric=metric, mode="test", n=n)
            clf = LDA()
            clf.fit(trainX, trainY)
            res = clf.score(testX, testY)
            av.append(res)
            print("Subject " + str(i) + ", accuracy = " + str(res))
        print("Average is " + str(sum(av)/len(av)) + "\n")

N is 2

Metric is classic
Subject 1, accuracy = 0.8
Subject 2, accuracy = 0.83
Subject 4, accuracy = 0.58
Subject 5, accuracy = 0.8
Subject 6, accuracy = 0.81
Subject 7, accuracy = 0.61
Subject 8, accuracy = 0.58
Subject 9, accuracy = 0.7
Subject 10, accuracy = 0.59
Subject 11, accuracy = 0.48
Subject 12, accuracy = 0.59
Subject 13, accuracy = 0.5
Subject 14, accuracy = 0.56
Subject 15, accuracy = 0.52
Subject 16, accuracy = 0.68
Subject 17, accuracy = 0.7
Subject 18, accuracy = 0.85
Subject 19, accuracy = 0.79
Subject 20, accuracy = 0.61
Average is 0.6621052631578946

Metric is AIRM
Subject 1, accuracy = 0.79
Subject 2, accuracy = 0.81
Subject 4, accuracy = 0.56
Subject 5, accuracy = 0.81
Subject 6, accuracy = 0.87
Subject 7, accuracy = 0.65
Subject 8, accuracy = 0.63
Subject 9, accuracy = 0.71
Subject 10, accuracy = 0.66
Subject 11, accuracy = 0.49
Subject 12, accuracy = 0.59
Subject 13, accuracy = 0.52
Subject 14, accuracy = 0.45
Subject 15, accuracy = 0.54
Subject 16, accuracy = 0.